In [33]:
## pip install opencv-contrib-python
## pip install opencv-python

In [36]:
import os
import json
import pandas as pd
import cv2

In [37]:
base_path = 'C:/Users/wjdgh/Desktop/Detection_Data/Training'
label_path = base_path + '/Labels_sample'
image_path = base_path + '/Images_sample'
crop_path = 'C:/Users/wjdgh/Desktop/Recognition_Data/Training/Images_sample'
csv_path = 'C:/Users/wjdgh/Desktop/Recognition_Data/Training/Labels_sample/info.csv'

In [38]:
image_items = sorted(os.listdir(image_path))
label_items = sorted(os.listdir(label_path))
info = pd.DataFrame(columns=['file_name','text'])

In [81]:
n_iter = len(image_items)

for i in range(n_iter):

    path_L = os.path.join(label_path,label_items[i])
    path_I = os.path.join(image_path,image_items[i])

    with open(path_L, 'r', encoding='UTF-8') as f:
        json_data = json.load(f)
    
    image = cv2.imread(path_I, cv2.IMREAD_COLOR)
    curr_image_name = json_data['Images']['identifier']

    if image is None:
        print("Error: Image not loaded.")
    else:
        for item in json_data['Bbox']:
            x = item['x']
            y = item['y']
            data = item['data']
            id = item['id']
            
            cropped_image = image[y[0]:y[3],x[0]:x[3]]
            cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
            cropped_image = cv.medianBlur(cropped_image,5)
            cropped_image = cv.adaptiveThreshold(cropped_image,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,\
                            cv.THRESH_BINARY,15,3)
            cropped_image = cv.fastNlMeansDenoising(cropped_image,None,10,7,21)


            name = curr_image_name + '_' + str(id) + '.jpg'
            info.loc[len(info.index)] = [name,data]
            crop_save_path = os.path.join(crop_path,name)
            cv2.imwrite(crop_save_path, cropped_image)

In [82]:
info.to_csv(csv_path, index=False, encoding='cp949')